In [4]:
%pip install torch pandas torchvision scikit-learn tqdm kaggle yacs timm einops munch -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
# upload kaggle.json first.
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [1]:
# !apt update -qq
# !apt install -qq unzip
# !kaggle datasets download nirmalsankalana/sugarcane-leaf-disease-dataset
# !unzip -q sugarcane-leaf-disease-dataset.zip -d data
!kaggle datasets download  pungliyavithika/sugarcane-leaf-disease-classification
!unzip -q sugarcane-leaf-disease-classification.zip -d data2

Dataset URL: https://www.kaggle.com/datasets/pungliyavithika/sugarcane-leaf-disease-classification
License(s): unknown


In [3]:
import os
import shutil
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Define base paths
data_root = "data2"
images_dir = os.path.join(data_root, "images")
os.makedirs(images_dir, exist_ok=True)

# Collect metadata for each image
dataset = []

for subfolder in os.listdir(data_root):
    subfolder_path = os.path.join(data_root, subfolder)

    # Skip non-directories and the images directory
    if not os.path.isdir(subfolder_path) or subfolder == "images":
        continue

    for image in os.listdir(subfolder_path):
        old_image_path = os.path.join(subfolder_path, image)

        if os.path.isfile(old_image_path):
            base, ext = os.path.splitext(image)
            new_base = f"{subfolder}_{base}"
            new_filename = f"{new_base}{ext}"
            new_image_path = os.path.join(images_dir, new_filename)

            # Ensure uniqueness
            counter = 1
            while os.path.exists(new_image_path):
                new_filename = f"{new_base}_{counter}{ext}"
                new_image_path = os.path.join(images_dir, new_filename)
                counter += 1

            # Move the file
            shutil.move(old_image_path, new_image_path)

            # Record the file metadata
            dataset.append({
                "image_id": new_filename,  # Just filename
                "label": subfolder,
                "original_filename": image
            })

    # Optionally remove the now-empty subfolder
    os.rmdir(subfolder_path)

# Create a DataFrame
df = pd.DataFrame(dataset)

# Encode the labels
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])

# Save the dataset to CSV
df.to_csv(os.path.join(data_root, "dataset.csv"), index=False)

# Save label mapping
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label mapping:", label_mapping)

Label mapping: {'Healthy': 0, 'RedRot': 1, 'RedRust': 2}


In [1]:
import pandas as pd
from dataset import build_dataset, build_loader

path = 'data2/images'
df = pd.read_csv('data2/dataset.csv')

trd, testd, vald = build_dataset(df, path)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_loader, test_loader, val_loader = build_loader(trd, vald, 16)

In [3]:
from train import main
model = main(train_loader, val_loader, 'tswinf_pungliyavithika.pth')

2025-04-27 20:17:14.343144: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-27 20:17:14.356523: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745785034.372303    6175 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745785034.377441    6175 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-27 20:17:14.393366: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

[2025-04-27 20:17:15 CSWin_BOAT_64_24322_small_224](train.py 71): INFO Creating model:cswin_boat/CSWin_BOAT_64_24322_small_224


/home/ubuntu/.local/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


[2025-04-27 20:17:27 CSWin_BOAT_64_24322_small_224](train.py 80): INFO number of params: 60235181
[2025-04-27 20:17:27 CSWin_BOAT_64_24322_small_224](train.py 101): INFO Start training
[2025-04-27 20:17:58 CSWin_BOAT_64_24322_small_224](train.py 205): INFO Train: [0/35]	eta 0:00:02 lr 0.000038	time 2.1827 (2.8675)	loss 1.5374 (1.5868)	grad_norm 2.0801 (1.4475)	mem 6408MB
[2025-04-27 20:17:58 CSWin_BOAT_64_24322_small_224](train.py 221): INFO EPOCH 0 training takes 0:00:31
[2025-04-27 20:18:02 CSWin_BOAT_64_24322_small_224](train.py 310): INFO TEST:	Time 1.827 (1.777)	Loss 1.5349 (1.5313)	Acc@1 37.500 (37.500)	Acc@5 100.000 (100.000)	Mem 6408MB
[2025-04-27 20:18:02 CSWin_BOAT_64_24322_small_224](train.py 318): INFO  * Acc@1 37.500 Acc@5 100.000
[2025-04-27 20:18:02 CSWin_BOAT_64_24322_small_224](train.py 322): INFO 
Classification Report:
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000        11
           1     0.0000    0.0000    0.00

/usr/lib/python3/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[2025-04-27 20:18:02 CSWin_BOAT_64_24322_small_224](train.py 124): INFO tswinf_pungliyavithika.pth saved !!!
[2025-04-27 20:18:02 CSWin_BOAT_64_24322_small_224](train.py 128): INFO Max accuracy: 37.50%
[2025-04-27 20:18:27 CSWin_BOAT_64_24322_small_224](train.py 205): INFO Train: [1/35]	eta 0:00:02 lr 0.000078	time 2.1217 (2.2590)	loss 1.4453 (1.5093)	grad_norm 3.1857 (2.7316)	mem 6408MB
[2025-04-27 20:18:27 CSWin_BOAT_64_24322_small_224](train.py 221): INFO EPOCH 1 training takes 0:00:24
[2025-04-27 20:18:31 CSWin_BOAT_64_24322_small_224](train.py 310): INFO TEST:	Time 2.044 (1.803)	Loss 1.4927 (1.4702)	Acc@1 31.250 (37.500)	Acc@5 100.000 (100.000)	Mem 6408MB
[2025-04-27 20:18:31 CSWin_BOAT_64_24322_small_224](train.py 318): INFO  * Acc@1 37.500 Acc@5 100.000
[2025-04-27 20:18:31 CSWin_BOAT_64_24322_small_224](train.py 322): INFO 
Classification Report:
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         9
           1     0.0000 

KeyboardInterrupt: 